<a href="https://colab.research.google.com/github/KAVYANSHTYAGI/Graph_Neural_Networks/blob/main/temporal_gnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install torch torchvision torch-geometric matplotlib networkx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 903.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [4]:
!pip install --upgrade torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-2.0.1+cu118.html

Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 886.5/886.5 kB 48.3 MB/s eta 0:00:00


In [5]:
import torch
import pandas as pd
import numpy as np
from torch_geometric.data import TemporalData
from torch_geometric.loader import TemporalDataLoader
from torch_geometric.nn import TGN, LastNeighborLoader
from torch_geometric.nn.models.tgn import IdentityMessage, LastAggregator

# 1. Load the Wikipedia dataset
url = "https://snap.stanford.edu/jodie/wikipedia.csv"
df = pd.read_csv(url, header=None, names=["user_id", "item_id", "timestamp", "user_feature", "item_feature"])


ImportError: cannot import name 'TGN' from 'torch_geometric.nn' (/usr/local/lib/python3.11/dist-packages/torch_geometric/nn/__init__.py)

In [ ]:
# 2. Normalize and preprocess data
num_nodes = max(df['user_id'].max(), df['item_id'].max()) + 1
df['user_feature'] = df['user_feature'].astype(float)
df['item_feature'] = df['item_feature'].astype(float)

df = df.sort_values(by='timestamp')

timestamps = torch.tensor(df['timestamp'].values, dtype=torch.long)
user_ids = torch.tensor(df['user_id'].values, dtype=torch.long)
item_ids = torch.tensor(df['item_id'].values, dtype=torch.long)
edges = torch.stack((user_ids, item_ids))

In [ ]:
# Convert features
def convert_features(series, num_nodes):
    features = torch.zeros((num_nodes, 1), dtype=torch.float)
    features[series.index, 0] = torch.tensor(series.values, dtype=torch.float)
    return features

node_features = convert_features(df['user_feature'], num_nodes)
edge_features = convert_features(df['item_feature'], num_nodes)

# 3. Create TemporalData object
data = TemporalData(
    src=user_ids,
    dst=item_ids,
    t=timestamps,
    msg=edge_features,
    y=torch.ones(user_ids.shape[0])  # Dummy labels for now
)

In [ ]:
# 4. Train-Test Split (80:20)
split_idx = int(0.8 * len(df))
train_data = data[:split_idx]
test_data = data[split_idx:]

# 5. Create Dataloaders
neighbor_loader = LastNeighborLoader(num_nodes=num_nodes, size=10)
train_loader = TemporalDataLoader(train_data, batch_size=64)
test_loader = TemporalDataLoader(test_data, batch_size=64)

In [ ]:

# 6. Define TGN Model
class SimpleTGN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.tgn = TGN(
            node_features=in_channels,
            edge_features=1,
            msg_fn=IdentityMessage(),
            agg_fn=LastAggregator(),
            neighbor_loader=neighbor_loader,
            memory_dim=32,
            time_enc_dim=32,
            num_layers=2
        )
        self.linear = torch.nn.Linear(32, out_channels)

    def forward(self, data):
        z = self.tgn(data.src, data.dst, data.t, data.msg)
        return self.linear(z)


In [ ]:

model = SimpleTGN(in_channels=1, out_channels=1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()


In [ ]:

# 7. Train the Model
def train():
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        pred = model(batch).squeeze()
        loss = criterion(pred, batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def test():
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in test_loader:
            pred = model(batch).squeeze()
            loss = criterion(pred, batch.y)
            total_loss += loss.item()
    return total_loss / len(test_loader)


In [ ]:

# Train for 10 epochs
for epoch in range(10):
    train_loss = train()
    test_loss = test()
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}")
